In [1]:
# 导包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell # 这个对象设置所有行全部输出
  
# 设置该对象ast_node_interactivity的属性值为all，表示notebook下每一行有输出的代码全部输出运算结果
InteractiveShell.ast_node_interactivity = "all"

# 解决坐标轴刻度负号乱码
plt.rcParams['axes.unicode_minus'] = False

# 解决中文乱码问题
plt.rcParams['font.sans-serif'] = ['Simhei']
plt.style.use('ggplot')



In [2]:
# 数据准备
row_data = {
  '是否陪伴' : [0, 0, 0, 1, 1],
   '是否玩游戏' : [1, 1, 0, 1, 1],
   '渣男' : ['是', '是', '不是', '不是', '不是']
}

# 构建为dataframe
dataset = pd.DataFrame(row_data)
dataset

,是否陪伴,是否玩游戏,渣男
0,0,1,是
1,0,1,是
2,0,0,不是
3,1,1,不是
4,1,1,不是


In [8]:
# 1.实现信息熵计算函数
def calEntropy(dataset):
  '''
  cal Entropy values
  '''
  
  # 先获取样本的总数
  n_samples = dataset.shape[0]
  
  #获取标签的所有类别
  classes = dataset.iloc[:, -1].value_counts()
  
  # 计算每一类标签占据样本总数的百分比
  px_i = classes / n_samples
  
  return (- px_i * np.log2(px_i)).sum() # 计算所得的信息熵

def test():
  print('当前输入数据的的信息熵为{}'.format(calEntropy(dataset)))

test()


当前输入数据的的信息熵为0.9709505944546686
